# Week 4 - Q&A

References:


*   [Langchain PDF loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)
*   [Langchain PDF splitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)
*   [Langchain text splitter](https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#langchain.text_splitter.RecursiveCharacterTextSplitter)
*   [Camelot: PDF Table Extraction for Humans](https://camelot-py.readthedocs.io/en/master/)
*   [competition](https://tianchi.aliyun.com/competition/entrance/532126/information)
*   [competition sample answer](https://github.com/RonaldJEN/FinanceChatGLM/tree/main)



## 0. Installation and Setup

In [ ]:
# hide output
%%capture output

! pip install pdfplumber
! pip install sentence-transformers
! pip install langchain
! pip install faiss-gpu
! pip install pypdf
! pip install layoutparser
! pip install pdfminer.six
! pip install unstructured
! pip install transformers
! pip install rapidocr-onnxruntime
! pip install pymupdf

In [ ]:
import os
from google.colab import drive
# Access drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Capstone/'


# companies
companies = os.listdir(os.path.join(path, 'Company Reports'))
for i, comp in enumerate(companies):
    print(i, ": ", comp)


# get reports
def get_reports(comp, year:int, rep_type:int = 1):
    """
    comp:       string or index
    year:       specific year or # recent year, 0 for all
    rep_type:   report type, 1 for annual report, 2 for sustainability report, 0 for both
    ret:        list of report pathes or a single report path
    """
    if type(comp) == str:
        if comp not in companies:
            print("Error: ", comp, " does not exist")
            return
    elif type(comp) == int:
        if comp not in range(len(companies)):
            print("Error: invalid index")
            return
        comp = companies[comp]
    else:
        print("Error: invalid company")
        return

    file_path = os.path.join(path, 'Company Reports', comp)
    files = os.listdir(file_path)
    files.sort(reverse=True)

    years = range(2013,2023)
    if year in range(11):
        if year:
            years = years[-year:]
    else:
        years = [year]

    if rep_type == 0:
        reps = ["", "_sus"]
    elif rep_type == 1:
        reps = [""]
    elif rep_type == 2:
        reps = ["_sus"]
    else:
        print("Error: invalid report type")
        return

    ret = []
    for year in years:
        for rep in reps:
            file = comp + '_' + str(year) + rep + '.pdf'
            if file in files:
                ret.append(file)

    ret_p = [os.path.join(file_path, file) for file in ret]
    if len(ret_p) == 1:
        return ret_p[0]
    else:
        return ret_p

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
0 :  ExxonMobil
1 :  Shell plc
2 :  BP PLC
3 :  Saudi Aramco
4 :  Chevron
5 :  TotalEnergies
6 :  Valero Energy
7 :  Marathon Petroleum Corporation
8 :  Sinopec
9 :  PetroChina


In [ ]:
file = get_reports(0, 2022)
file

'/content/drive/MyDrive/Capstone/Company Reports/ExxonMobil/ExxonMobil_2022.pdf'

## 1. Load Data
In Langchiain, we use document_loaders to load our data. We can simply import langchain.document_loaders and specify the data type.
1. folder: DirectoryLoader
2. Azure: AzureBlobStorageContainerLoader
3. CSV file: CSVLoader
4. Google Drive: GoogleDriveLoader
5. Website: UnstructuredHTMLLoader
6. PDF: PyPDFLoader
7. Youtube: YoutubeLoader

For more data loader refer to the following link:
https://python.langchain.com/docs/modules/data_connection/document_loaders.html

### PyPDFium2

In [ ]:
from langchain.document_loaders import PyPDFium2Loader

loader = PyPDFium2Loader(file)

## 2. Split the data
Once we loaded documents, we need to transform them to better suit our application. The simplest example is to split a long document into smaller chunks that can fit into our model's context window. The most common Splitter in LangChain includes:

1. RecursiveCharacterTextSplitter()
2. CharacterTextSplitter()

The paramether of above functions:
 - length_function: how the length of chunks is calculated. Defaults to just counting number of characters, but it's pretty common to pass a token counter here.
 - chunk_size: the maximum size of your chunks (as measured by the length function).
 - chunk_overlap: the maximum overlap between chunks. It can be nice to have some overlap to maintain some continuity between chunks (e.g. do a sliding window).
 - add_start_index: whether to include the starting position of each chunk within the original document in the metadata.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", " ", "", "."],
    chunk_size = 1000,
    chunk_overlap = 300
)

## 3. Vectorstore

In [ ]:
text_splitter.split_documents(loader.load())

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings()

data = text_splitter.split_documents(loader.load())

vs = FAISS.from_documents(data, embeddings)

## 4. Model

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM, AutoModelForCausalLM

model_id_mistral = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer_mistral = AutoTokenizer.from_pretrained(model_id_mistral)
model_mistral = AutoModelForCausalLM.from_pretrained(model_id_mistral)

pipe_mistral = pipeline(
    "text-generation",
    model = model_mistral,
    tokenizer = tokenizer_mistral,
    max_length = 1500,
    pad_token_id = model_mistral.config.eos_token_id
)

llm_mistral = HuggingFacePipeline(pipeline = pipe_mistral)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 5. Testing

In [ ]:
import time
from langchain.chains.question_answering import load_qa_chain

def print_doc(q, a):
    s = 'PyPDFium2 + rec_1500_300'
    print('-'*100)
    print('|', s, '|')
    print('-'*32)
    print(q)
    for i, d in enumerate(vs.similarity_search(q)):
        print('-'*100)
        if a in d.page_content:
            found = 'Found: \x1b[31mTrue\x1b[0m'
        else:
            found = 'Found: False'

        if 'page' in d.metadata:
            print('|', str(i+1)+'. Page', d.metadata['page']+1, '|', found, '|')
        else:
            print('|', str(i+1), '|', found, '|')
        #print('|', str(i+1)+'. Page |')
        print('-'*32)
        print(d.page_content)
    print('-'*100)


def get_answer(q, vs, r, llm):
    s1 = time.time()
    if vs:
        doc = vs.similarity_search(q)
    else:
        doc = r.get_relevant_documents(q)
    t1 = time.time() - s1

    s2 = time.time()
    chain = load_qa_chain(llm, chain_type="stuff")
    res = chain({"input_documents": doc, "question": q}, return_only_outputs=True)
    t2 = time.time() - s2
    return res['output_text'], round(t1, 2), round(t2, 2)


def show_results(q):
    print("-" * 100)
    print("| FAISS + Mistral |")
    print("-" * 100)
    print("    - Q:", q)
    res, t1, t2 = get_answer(q, vs, None, llm_mistral)
    print("    - A:", res)
    print("         retriver time: ", t1, 's')
    print("         model time:    ", t2, 's')
    print("-" * 100)

### 5.1 Q1

In [ ]:
# Answer at page 14
q = 'What is ExxonMobil’s worldwide environmental expenditures in 2022?'
a = '5.7'
print_doc(q, a)

----------------------------------------------------------------------------------------------------
| PyPDFium2 + rec_1500_300 |
--------------------------------
What is ExxonMobil’s worldwide environmental expenditures in 2022?
----------------------------------------------------------------------------------------------------
| 1. Page 14 | Found: True |
--------------------------------
in refining infrastructure and technology to manufacture clean fuels, as well as projects to monitor and 
reduce air, water, and waste emissions, and expenditures for asset retirement obligations. Using definitions and guidelines established 
by the American Petroleum Institute, ExxonMobil' s 2022 worldwide environmental expenditures for all such preventative and 
remediation steps, including ExxonMobil's share of equity company expenditures, were $5.7 billion, of which $3.8 billion were 
included in expenses with the remainder in capital expenditures. As the Corporation progresses its emission-reduc

In [ ]:
show_results(q)

----------------------------------------------------------------------------------------------------
| FAISS + Mistral |
----------------------------------------------------------------------------------------------------
    - Q: What is ExxonMobil’s worldwide environmental expenditures in 2022?
    - A:  ExxonMobil's worldwide environmental expenditures in 2022 were $5.7 billion.
         retriver time:  0.03 s
         model time:     57.18 s
----------------------------------------------------------------------------------------------------


### 5.2 Q2

In [ ]:
# Answer at page 142
q = 'What is ExxonMobil’s Future production cost in Europe?'
a = '1,815'
print_doc(q, a)

----------------------------------------------------------------------------------------------------
| PyPDFium2 + rec_1500_300 |
--------------------------------
What is ExxonMobil’s Future production cost in Europe?
----------------------------------------------------------------------------------------------------
| 1. Page 149 | Found: False |
--------------------------------
Source: ExxonMobil analysis of EPA Facility Level Information on Greenhouse Gases Tool, 2019 data as of Feb. 15, 2022.
17. Statements of potential future earnings and cash flow assume $60/bbl Brent crude prices and $3/mmbtu Henry Hub gas
prices, adjusted for inflation from 2022; Energy, Chemical, and Specialty Product margins at historical averages for the 10-
year period from 2010-2019; and before tax Corporate & Financing expenses between $2.3 and $2.5 billion annually. 2019
baseline excludes identified items. Price assumptions are not intended to reflect management’s forecasts for future prices or
the price

## 6.Table extraction

In [ ]:
file

'/content/drive/MyDrive/Capstone/Company Reports/ExxonMobil/ExxonMobil_2022.pdf'

### Tabula

In [ ]:
! pip install tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 99.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 42.0 MB/s eta 0:00:00


In [ ]:
from tabula import read_pdf

tables = read_pdf(file, pages='all')

In [ ]:
len(tables)

6

In [ ]:
tables[0]

,E X X O N M O B I L,C O R P O R A T I O N,|,2 0 2 2,A N N U A L,R E P O R T,Unnamed: 0
0,2022 : A ye a r in r ev i ew,NaN,NaN,NaN,NaN,NaN,NaN
1,Financial and operating performance significan...,NaN,NaN,NaN,NaN,NaN,NaN
2,Earnings,NaN,NaN,NaN,Cash flow from operations,NaN,Structural cost savings 2
3,$56B,NaN,NaN,NaN,$77B,NaN,$7B
4,25%,NaN,NaN,NaN,$30B,NaN,87%
5,ROCE with $23B in capex3,NaN,NaN,NaN,shareholder distributions,NaN,total shareholder return4
6,Continuing to be an industry leader in safety5,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Working to solve the “and” equation:,NaN,NaN,NaN,NaN,NaN
8,NaN,delivering the energy and products society needs,NaN,NaN,NaN,NaN,NaN
9,and reducing our own and others’ greenhouse ga...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
tables[1]

,Unnamed: 0,I I I
0,Up s tre a m,NaN
1,Our Upstream business works each day to provid...,NaN
2,"We continuously innovate and invest, using ind...",NaN
3,increase oil and natural gas production to mee...,NaN
4,population.,NaN
5,"In 2022, we continued to strengthen and active...",NaN
6,strategic projects while producing 3.7 million...,NaN
7,"Basin, we maximized the value of our large acr...",NaN
8,"of our integrated operations, increasing produ...",NaN
9,per day year-over-year while making progress o...,NaN


In [ ]:
tables[2]

,Unnamed: 0,V I I
0,L ow C a r b o n S o l u t i o ns,NaN
1,2022 was a milestone year for Low Carbon Solut...,NaN
2,gas emissions from our operations and provides...,NaN
3,customers’ emissions. We do this through a foc...,NaN
4,and lower-emission fuels.,NaN
5,These three technologies fit well with our pro...,NaN
6,advantages. They are also critical to reducing...,NaN
7,"decarbonize sectors: manufacturing, power gene...",NaN
8,strategy gives us the flexibility to pace inve...,NaN
9,as markets and policies evolve. With clear and...,NaN


In [ ]:
tables[3]

,V I I I E X X O N,M O B I L,C O R P O R A T I O N,|,2 0 2 2,A N N U A L,R E P O R T,Unnamed: 0
0,O u r w in n i n g pr o p o s i t i o n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Corporate plan through 2027,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Our unique competitive advantages have been bu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,through the right strategic priorities and ext...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,plan is expected to drive leading business out...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,performance in 2022.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,2X,NaN,NaN,NaN,NaN,20-30%
7,earnings and cash flow growth,NaN,NaN,NaN,NaN,NaN,NaN,reduction in corporate-wide
8,potential by 2027 versus 201917,NaN,NaN,NaN,NaN,NaN,NaN,greenhouse gas intensity by 203018
9,$20-25B,NaN,NaN,NaN,NaN,NaN,NaN,Up to $35B


In [ ]:
tables[4]

,Unnamed: 0,Securities registered pursuant to Section 12(b) of the Act:,Unnamed: 1
0,Title of Each Class,Trading Symbol,Name of Each Exchange on Which Registered
1,"Common Stock, without par value",XOM,New York Stock Exchange
2,0.142% Notes due 2024,XOM24B,New York Stock Exchange
3,0.524% Notes due 2028,XOM28,New York Stock Exchange
4,0.835% Notes due 2032,XOM32,New York Stock Exchange
5,1.408% Notes due 2039,XOM39A,New York Stock Exchange


In [ ]:
tables[5]

,"For the fiscal year ended December 31, 2022"
0,or
1,□ TRANSITION REPORT PURSUANT TO SECTION 13 OR ...
2,THE SECURITIES EXCHANGE ACT OF 1934
3,For the transition period from ___ to ___
4,Commission File Number 1-2256
5,Exxon Mobil Corporation
6,(Exact name of registrant as specified in its ...
7,New Jersey 13-5409005
8,(State or other jurisdiction of (I.R.S. Employer
9,incorporation or organization) Identification ...


### look into a particular page, page 47

try tabula

In [ ]:
page = '/content/page_47.pdf'

In [ ]:
from tabula import read_pdf
t = read_pdf(page, pages='all')

In [ ]:
t

[]

try PyPDFLoader

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(page)
pages = loader.load_and_split()

In [ ]:
pages

[]

### look into all the parsed pages

In [ ]:
s = []

for d in data:
    s.append(d.metadata['page'])

miss = []
for i in range(152):
    if i not in s:
        miss.append(i)

print('Missing', len(miss),'out of 152 pages: ')
ss = ''
idx = 0
for i in miss:
    if idx % 10 == 0:
        ss += '\n'
    ss += str(i+1) + ', '
    idx += 1
print(ss)

Missing 121 out of 152 pages: 

15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 
26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 
36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 
46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 
56, 57, 58, 59, 60, 61, 64, 65, 68, 71, 
74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 
84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 
94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 
104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 
114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 
124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 
134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 
144, 


In [ ]:
def get_page(x):
    ret = []
    for d in data:
        if d.metadata['page'] == x-1:
            ret.append(d.page_content)

    for s in ret:
        print(s)

In [ ]:
get_page(14)

As discussed in item IA. Risk Factors in this report, compliance with existing and potential future government regulations, including 
taxes, environmental regulations, and other government regulations and policies that directly or indirectly affect the production and 
sale of our products, may have material effects on the capital expenditures, earnings, and competitive position of ExxonMobil. With 
respect to the environment, throughout ExxonMobil' s businesses, new and ongoing measures are taken to prevent and minimize the 
impact of our operations on air, water, and ground, including, but not limited to, compliance with environmental regulations. These 
include a significant investment in refining infrastructure and technology to manufacture clean fuels, as well as projects to monitor and 
reduce air, water, and waste emissions, and expenditures for asset retirement obligations. Using definitions and guidelines established 
by the American Petroleum Institute, ExxonMobil' s
in refin

In [ ]:
get_page(15)

In [ ]:
get_page(16)

### try other parsers

In [ ]:
from pdfminer import high_level

In [ ]:
pages = [0]
high_level.extract_text(page, "", pages)

'\x0c'

### missing percentage

In [ ]:
import PyPDF2
miss_dic = {}

for i, comp in enumerate(companies):
    rep = get_reports(i, 0)
    lens = []
    miss = []
    for r in rep:
        l = len(PyPDF2.PdfReader(r).pages)
        lens.append(l)
        data_tmp = text_splitter.split_documents(PyPDFium2Loader(r).load())
        s = []
        m = []

        for d in data_tmp:
            s.append(d.metadata['page'])

        for i in range(l):
            if i not in s:
                m.append(i)
        miss.append(len(m))
    miss_dic[comp] = sum(miss) / sum(lens)

In [ ]:
import pandas as pd
pd.DataFrame.from_dict(miss_dic, orient='index', columns = ['Missing Percentage'])

,Missing Percentage
ExxonMobil,0.216458
Shell plc,0.005263
BP PLC,0.001088
Saudi Aramco,0.002066
Chevron,0.000000
TotalEnergies,0.001425
Valero Energy,0.001957
Marathon Petroleum Corporation,0.004376
Sinopec,0.000000
PetroChina,0.016102
